In [30]:
import imp
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from loss import *
from torch.autograd import Variable
import copy
from metrice import *

# -----model hyperparameters-------
# temperature in consistency constraint loss
temperature = 1
# the weight of consistency constraint loss
alpha = 2


def test(dataloader, model , device = "cpu"):
    # switch to eval mode
    model.eval()# switch to eval mode
    
    loss_Focalloss = FocalLoss(gamma=2)
    loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
    losses = []
    
    loader = tqdm(dataloader)
    iou = 0

    with torch.no_grad():
        # for idx, (img, mask, mask_boundary) in enumerate(loader):
        for i, (input_ori, input, edge, mask) in enumerate(loader):
            # Send to device
            input_ori = input_ori.to(device)
            input = input.to(device)
            mask = mask.to(device)
            edge = edge.to(device)

            # load input to model
            output_mask_texture, output_edge_texture = model(input)
            output_mask_deformation, output_edge_deformation = model(input_ori)

            # mask loss and edge loss
            loss_mask_texture = loss_Softmax(output_mask_texture, mask)
            loss_edge_texture = loss_Focalloss(output_edge_texture, edge) * 0.1
            loss_mask_deformation = loss_Softmax(output_mask_deformation, mask)
            loss_edge_deformation = loss_Focalloss(output_edge_deformation, edge) * 0.1

            # consistency constraint loss: KL distance
            loss_kl_mask = loss_KL(output_mask_texture, Variable(output_mask_deformation.data, requires_grad=False),
                                temperature) * alpha
            loss_kl_edge = loss_KL(output_edge_texture, Variable(output_edge_deformation.data, requires_grad=False),
                                temperature) * alpha * 0.1

            #total loss
            # loss = loss_mask_texture + loss_edge_texture + loss_mask_deformation + loss_edge_deformation + loss_kl_mask + loss_kl_edge
            # loss = loss_mask + loss_mask_ori + loss_kl_mask + loss_edge
            loss = loss_mask_texture + loss_mask_deformation + loss_kl_mask + loss_edge_texture
                
            losses.append(loss.item())

            softmax = nn.Softmax(dim=1)
            prob = softmax(output_mask_texture)[0,1,:,:]
            pred = prob.data.cpu().numpy()
            pred[pred>0.5] = 1
            pred[pred<=0.5] = 0
            iou += calcIOU(pred, mask[0].data.cpu().numpy())

    return sum(losses)/len(losses),  iou / len(loader)

In [31]:
import torch
import torch.nn as nn
from tqdm import tqdm
from loss import *
from torch.autograd import Variable
from metrice import *

batch_size = 64

epoches = 2000

weight_decay = 5e-4

learning_rate = 1e-3

lr_decay = 0.95

lr_decay_epoch = epoches // 20


# -----model hyperparameters-------
# temperature in consistency constraint loss
temperature = 1
# the weight of consistency constraint loss
alpha = 2

# def adjust_learning_rate(optimizer, epoch, multiple):
#     """Sets the learning rate to the initial LR decayed by 0.95 every 20 epochs"""
#     lr = learning_rate * (lr_decay ** (epoch // 20))
#     for i, param_group in enumerate(optimizer.param_groups):
#         param_group['lr'] = lr * multiple[i]
#     pass


def train(dataloader, model, optimizer, device="cpu"):
    model.train()  # switch to train mode
    loss_Softmax = nn.CrossEntropyLoss(ignore_index=255) # mask loss
    loss_Focalloss = FocalLoss(gamma=2) # boundary loss

    losses = []
    iou = 0
    loader = tqdm(dataloader)

    # for i, (input_deformation, input_texture, mask, mask_boundary) in enumerate(loader):
    for i, (input_deformation, input_texture, mask_boundary, mask) in enumerate(loader):

        #Send to device
        input_deformation = input_deformation.to(device)
        input_texture = input_texture.to(device)
        mask = mask.to(device)
        mask_boundary = mask_boundary.to(device)

        #load input to model
        output_mask_deformation, output_edge_deformation = model(input_deformation)
        output_mask_texture, output_edge_texture = model(input_texture)

        #mask loss and edge loss
        loss_mask_texture = loss_Softmax(output_mask_texture,mask)
        loss_edge_texture = loss_Focalloss(output_edge_texture,mask_boundary)*0.1
        loss_mask_deformation = loss_Softmax(output_mask_deformation,mask)
        loss_edge_deformation = loss_Focalloss(output_edge_deformation,mask_boundary)*0.1
       
       # consistency constraint loss: KL distance
        loss_kl_mask = loss_KL(output_mask_texture, Variable(output_mask_deformation.data, requires_grad=False),
                              temperature) * alpha
        loss_kl_edge = loss_KL(output_edge_texture, Variable(output_edge_deformation.data, requires_grad=False),
                              temperature) * alpha * 0.1

        #total loss
        # loss = loss_mask_texture + loss_edge_texture + loss_mask_deformation + loss_edge_deformation + loss_kl_mask + loss_kl_edge
        # loss = loss_mask + loss_mask_ori + loss_kl_mask + loss_edge
        loss = loss_mask_texture + loss_mask_deformation + loss_kl_mask + loss_edge_texture
        
        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loader.set_postfix(loss=loss.item())

        losses.append(loss.item())

        softmax = nn.Softmax(dim=1)
        prob = softmax(output_mask_texture)[0,1,:,:]
        pred = prob.data.cpu().numpy()
        pred[pred>0.5] = 1
        pred[pred<=0.5] = 0
        iou += calcIOU(pred, mask[0].data.cpu().numpy())

    
    return sum(losses)/len(losses), iou / len(dataloader)

In [32]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # -------use official dataset definition--------------
    exp_args = edict()
    exp_args.istrain = True
    exp_args.task = 'seg'
    exp_args.datasetlist =['EG1800']
    exp_args.data_root = './'
    exp_args.file_root = './EG1800'

    exp_args.input_height = 224
    exp_args.input_width = 224


    exp_args.prior_prob = 0.5 # the probability to set empty prior channel

    exp_args.edgeRatio = 0.1 # the weight of boundary auxiliary loss
    # exp_args.stability = True
    exp_args.temperature = 1 # the temperature in consistency constraint loss, default=1
    exp_args.alpha = 2 # the weight of consistency constraint loss, default=2
    ############################
    exp_args.padding_color=128 # input normalization parameters
    exp_args.img_scale = 1
    exp_args.img_mean = [103.94, 116.78, 123.68] # BGR order, image mean
    exp_args.img_val = [0.017, 0.017, 0.017] # BGR order, image val, default=[1/0.017, 1/0.017, 1/0.017]
    ##########################
    exp_args.init = False # whether to use pretrained model to init portraitnet
    exp_args.resume = False # whether to continue training

    # set training dataset
    exp_args.learning_rate = 1e-3
    exp_args.momentum = 0.9
    exp_args.weight_decay = 5e-4
    exp_args.batch_size = 1
    #######################下面没什么用
    exp_args.addEdge = True
    exp_args.stability = True
    exp_args.use_kl = True
    exp_args.useUpsample = False
    exp_args.useDeconvGroup = False
    exp_args.video = False
    exp_args.istrain = True

    dataset_train = Human(exp_args)
    dataLoader_train = torch.utils.data.DataLoader(dataset_train, batch_size=64,
                                                   shuffle=True)

    # set testing dataset
    exp_args.istrain = False
    dataset_test = Human(exp_args)
    dataLoader_test = torch.utils.data.DataLoader(dataset_test, batch_size=64,
                                                  shuffle=False)
    #---------------------------
    #Get model
    portrait_net = ProtraitNet(n_class=2)

#     #use some weights
#     pretrained_mobilenet_dict = torch.load("./mobilenet_v2-b0353104.pth")
#     pretrained_mobilenet_dict_keys = list(pretrained_mobilenet_dict.keys())
#     portrait_net_dict = portrait_net.state_dict()
#     portrait_net_dict_keys = list( portrait_net.state_dict().keys())
#     print ("pretrained mobilenetv2 keys: ", len(pretrained_mobilenet_dict))
#     print ("portraitnet keys: ", len(portrait_net_dict_keys))
#     weights_load = {}
#     for k in range(len(pretrained_mobilenet_dict_keys)):
#         if pretrained_mobilenet_dict[pretrained_mobilenet_dict_keys[k]].shape == \
#         portrait_net_dict[portrait_net_dict_keys[k]].shape:
#             weights_load[portrait_net_dict_keys[k]] = pretrained_mobilenet_dict[pretrained_mobilenet_dict_keys[k]]
#             print ('init model', portrait_net_dict_keys[k],
#                    'from pretrained', portrait_net_dict_keys[k])
#         else:
#             break
#     print ("init len is:", len(weights_load))
#     portrait_net_dict.update(weights_load)
#     portrait_net.load_state_dict(portrait_net_dict)
#     print ("load model init finish...")
    portrait_net.to(device)

    #Get optimizer
    optimizer = torch.optim.Adam(portrait_net.parameters(), lr=learning_rate, weight_decay=weight_decay)

    #Get Lr scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,lr_decay_epoch,lr_decay)

    current_epoch = 0
    min_val_loss = 10000
    train_losses = []
    test_losses = []
    test_miou = 0

    #Get optimizer
    for g in optimizer.param_groups:
        g['lr']=1e-3

    best_val_iou = 0
    # minLoss = 10000

    for epoch in range(current_epoch, epoches):
        print("Epoch : {}/{} , learning rate : {}".format(epoch,epoches,optimizer.param_groups[0]['lr']))
        train_loss, train_iou = train(dataLoader_train,portrait_net,optimizer,device)
         # Print the information.
        print(f"[ Train | {epoch:03d}/{epoches:03d} ] train_loss = {train_loss:.5f}, train_iou = {train_iou:.5f}")

        test_loss, test_iou = test(dataLoader_test,portrait_net,device)
        # Print the information.
        print(f"[ Valid | {epoch:03d}/{epoches:03d} ] test_loss  = {test_loss:.5f}, test_loss = {test_loss:.5f}")

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
#         if test_loss < minLoss:
#             minLoss = test_loss
#             print(f"[ Valid | {epoch:03d}/{epoches:03d} ] test_loss = {test_loss:.5f} -> best")
#             print(f"Best model found at epoch {epoch}, saving model")
#             torch.save(portrait_net.state_dict(), f"./myPortraitNet.ckpt") # only save best to prevent output memory exceed error


         # update logs
        if test_iou > best_val_iou:
            with open(f"./log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{epoches:03d} ] test_loss = {test_loss:.5f}, test_iou = {test_iou:.5f} -> best")
        else:
            with open(f"./log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{epoches:03d} ] test_loss = {test_loss:.5f}, test_iou = {test_iou:.5f}")

        # Save best model
        if test_iou > best_val_iou:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(portrait_net.state_dict(), f"./myPortraitNet.ckpt") # only save best to prevent output memory exceed error
            best_val_iou = test_iou

        print("Train Loss : {}, Test Loss : {} , Test IOU : {} , best Test mIOU : {}".format(train_loss,test_loss,test_iou,best_val_iou))
        lr_scheduler.step()

In [33]:
main()

Epoch : 0/2000 , learning rate : 0.001


100%|██████████| 23/23 [00:32<00:00,  1.41s/it]


[ Train | 000/2000 ] train_loss = 1.83389, train_iou = 0.42277


100%|██████████| 5/5 [00:05<00:00,  1.02s/it]


[ Valid | 000/2000 ] test_loss  = 1.59279, test_loss = 1.59279
[ Valid | 001/2000 ] test_loss = 1.59279, test_iou = 0.41954 -> best
Best model found at epoch 0, saving model
Train Loss : 1.833890816439753, Test Loss : 1.5927927494049072 , Test IOU : 0.4195437102128494 , best Test mIOU : 0.4195437102128494
Epoch : 1/2000 , learning rate : 0.001


...

[ Train | 184/2000 ] train_loss = 0.15161, train_iou = 0.93948


100%|██████████| 5/5 [00:04<00:00,  1.02it/s]


[ Valid | 184/2000 ] test_loss  = 0.14279, test_loss = 0.14279
[ Valid | 185/2000 ] test_loss = 0.14279, test_iou = 0.97068
Train Loss : 0.15160634336264237, Test Loss : 0.14278926253318786 , Test IOU : 0.9706773047966498 , best Test mIOU : 0.9715694644737967
Epoch : 185/2000 , learning rate : 0.00095


 65%|██████▌   | 15/23 [00:20<00:10,  1.36s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure()                   # 设置图片信息 例如：plt.figure(num = 2,figsize=(640,480))
y_train_loss = train_losses        # loss值，即y轴
x_train_loss = range(len(y_train_loss))
plt.plot(x_train_loss, y_train_loss, linewidth=1, linestyle="solid", label="train loss")        # epoch_losses 传入模型训练中的 loss[]列表,在训练过程中，先创建loss列表，将每一个epoch的loss 加进这个列表
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()        #个性化图例（颜色、形状等）
plt.savefig("./train_loss.jpg")